# cvairs
## Credit valuation adjustment (CVA) for interest rate swap (IRS)
Calculate CVAs for counterparties entering an IRS.

In [1]:
import csv

In [2]:
from cvairs import CCR, CVA, IRSExposure, VasicekModel

In [3]:
def params_from_csv(file):
    with open(file, newline='') as params_file:
        reader = csv.reader(params_file)
        next(reader)  # Skip main params header.
        params = []  # [Value: float]
        for row in reader:
            if not row[1]:
                break
            params.append(float(row[1]))
        next(reader)  # Skip credit curve header.
        # [(Time: years, Credit spread: bps)]
        credit_curve = [(float(row[0]), float(row[1])) for row in reader]
        return params, credit_curve

In [4]:
params, credit_curve = params_from_csv('../data/params.csv')
dt, LGD = params[5], params[11]

### Vašíček model
Generate short‑term interest rates to calculate zero‑coupon bond prices using the discrete‑time Vašíček model.

In [5]:
model = VasicekModel(*params[:9])

### Interest rate swap (IRS) exposure
Simulate swap market‑to‑market values estimate expected and potential future exposures of a counterparty entering an IRS (as a fixed pay receiver by default).

In [6]:
exposure = IRSExposure(*params[9:11], model)
mtms = exposure.market_to_market_simulations()
EE = exposure.expected_exposures(mtms)

### Counterparty credit risk (CCR)
Interpolate a credit curve using cubic spline to calculate probabilities of default.

In [7]:
x, y = zip(*credit_curve)
t, S = CCR.cubic_spline(x, y, dt)
PD = CCR.default_probabilities(t, S, LGD)

### Credit value adjustment (CVA)
Calculate a CVA for a counterparty entering an IRS.

In [8]:
cva = CVA.cva(LGD, EE, PD)
risky_duration = CVA.risky_duration(S, LGD)
cva_bps_approx1 = CVA.cva_bps_approx1(cva, risky_duration)
epe = CVA.epe(EE)
cva_bps_approx2 = CVA.cva_bps_approx2(epe, S)
cva_values = [('CVA', cva), ('Risky Duration', risky_duration),
              ('CVA (bps) Approximation 1', cva_bps_approx1), ('EPE', epe),
              ('CVA (bps) Approximation 2', cva_bps_approx2)]

cva_values

[('CVA', -0.004597763302919525),
 ('Risky Duration', 7.7858195152470735),
 ('CVA (bps) Approximation 1', -5.905304244358175),
 ('EPE', 0.018959879414417808),
 ('CVA (bps) Approximation 2', -5.953402136127192)]

In [9]:
headers = ['Time', 'Interp. credit curve', 'Prob default', 'EE']
row_widths = [len(header) for header in headers]
print(*headers)
print(*[f'{t[i]:<{row_widths[0]}} {S[i]:<{row_widths[1]}.10f} {PD[i]:<{row_widths[2]}.10f} {EE[i]:.10f}'
        for i in range(len(t))], sep='\n')

Time Interp. credit curve Prob default EE
0.0  0.0000000000         0.0000000000 0.0035155340
0.25 58.1352690077        0.0024193715 0.0122921493
0.5  113.0164304124       0.0069544536 0.0162298611
0.75 161.3893766108       0.0105977200 0.0185816937
1.0  200.0000000000       0.0128123545 0.0209179393
1.25 226.4810124034       0.0133038041 0.0218192094
1.5  242.0124033505       0.0126214462 0.0232771328
1.75 248.6609817977       0.0112493963 0.0248047969
2.0  248.4935567010       0.0095349251 0.0253862688
2.25 243.5769370168       0.0078004247 0.0268820313
2.5  235.9779317010       0.0063510324 0.0271761748
2.75 227.7633497101       0.0054825668 0.0272250876
3.0  221.0000000000       0.0054861748 0.0276324591
3.25 217.3188426224       0.0064364058 0.0276754193
3.5  216.6074420103       0.0076474147 0.0274468444
3.75 218.3175136920       0.0088512123 0.0272388207
4.0  221.9007731959       0.0099631297 0.0266128960
4.25 226.8089360503       0.0109034825 0.0260493614
4.5  232.4937177835   